In [1]:
import numpy as np
import joblib
import warnings 
warnings.simplefilter('ignore')
from sklearn.metrics import r2_score ,mean_squared_error
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from sklearn.svm import SVR

def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# data

In [2]:
# all data
data = joblib.load('../data/dataset.pkl')
b = data['stream']
A = np.hstack((data['action'].reshape(len(b),-1),data['state']))

batch_size = b.shape[0]

print(A.shape,b.shape)

(126443, 398) (126443, 1)


In [3]:
# split data
sp1 = -10000
sp2 = -1585
A_train = A[sp1:sp2]
b_train = b[sp1:sp2]
A_test = A[sp2:]
b_test = b[sp2:]

# Lasso find w

In [4]:
def evaluate(y_real,y_pred,name):
    score = pd.DataFrame(index = ['表現'],columns = ['R2','MSE','MAPE'])
    score['R2'] = r2_score(y_real,y_pred)
    score['MSE'] = mean_squared_error(y_real,y_pred)
    score['MAPE'] = mape(y_real,y_pred)
    score.index.name = name
    return score

In [5]:
params = {}
s = 9.000000e-07
t = 9.000000e-07 * 0.1
e = s + 10*t
#for a in tqdm(np.arange(s,e,t)):
#    model = Lasso(a).fit(A_train,b_train)
#    y_pred = data['mm_stream'].inverse_transform(model.predict(A_test).reshape(-1,1))
#    y_real = data['mm_stream'].inverse_transform(b_test)
#    params[a] = evaluate(y_real,y_pred,'bababa')

In [6]:
temp = pd.DataFrame([ i.R2.values[0] for i in params.values()],index=params.keys(),columns=['R2'])
temp.index.name = 'lambda'
temp

,R2
lambda,


In [7]:
model = Lasso(9.000000e-07).fit(A_train,b_train)

In [8]:
data['action_col']

['MLPAP_FQ-0619.PV',
 'MLPAP_TJC-0627.PV',
 'MLPAP_TRC-0610.PV',
 'MLPAP_FR-0632.PV',
 'MLPAP_TJC-0757A.PV',
 'MLPAP_PIC-0761.PV',
 'MLPAP_TJC-1721.PV',
 'MLPAP_TRC-1734.PV',
 'MLPAP_TRC-0747.PV',
 'MLPAP_TJ-0757B.PV',
 '濃度']

In [9]:
data['action'].shape

(126443, 36, 11)

In [10]:
action = data['mm_action'].inverse_transform(data['action'].reshape(126443*36,11))
feed = action[:,0]
feed = feed[::36]
feed.shape

(126443,)

In [11]:
y_pred = data['mm_stream'].inverse_transform(model.predict(A_train).reshape(-1,1)).reshape(-1)
y_real = data['mm_stream'].inverse_transform(b_train).reshape(-1)
evaluate(feed[:8415]/y_real,feed[:8415]/y_pred,'train(蒸氣單耗)')

,R2,MSE,MAPE
train(蒸氣單耗),,,
表現,0.999174,5.742558e-07,0.249739


In [12]:
y_pred = data['mm_stream'].inverse_transform(model.predict(A_test).reshape(-1,1)).reshape(-1)
y_real = data['mm_stream'].inverse_transform(b_test).reshape(-1)
evaluate(feed[-1585:]/y_real,feed[-1585:]/y_pred,'test(蒸氣單耗)')

,R2,MSE,MAPE
test(蒸氣單耗),,,
表現,0.856063,0.000002,0.520906


In [13]:
joblib.dump(model.coef_,'../model/stream_lasso_weight.pkl')

['../model/stream_lasso_weight.pkl']